 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="#IC" data-toc-modified-id="IC-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>IC</a></span></li><li><span><a href="#Calculate-correlation-with-H3K27ac" data-toc-modified-id="Calculate-correlation-with-H3K27ac-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Calculate correlation with H3K27ac</a></span></li></ul></div>

Try to run motifbreakR (https://www.bioconductor.org/packages/devel/bioc/vignettes/motifbreakR/inst/doc/motifbreakR-vignette.html#fn3) on enhancer SNPs.

In [1]:
import pandas as pd
from statsmodels.sandbox.stats.multicomp import multipletests
from scipy.stats import spearmanr

In [2]:
# Intergenic enhancer variants
snps = pd.read_csv('out_files/enhancer.SNPs.hard_filtered.VAF-cov_filtered.csv.intergenic.3samples-per-GT.csv', sep = '\t')

snps.head()

,CHROM,POS,REF,ALT,chr,start,end,GB08.act,GB02.act,GB01.act,...,GB15.var,PA01.var,PA02.var,PA04.var,PA05.var,PA06.var,PG11.var,Num homref,Num het,Num homalt
0,chr1,2966572,G,T,chr1,2966403,2970363,0.985579,1.114378,1.149652,...,1,1,0,1,1,1,2,3,11,11
1,chr1,2969791,A,G,chr1,2966403,2970363,0.985579,1.114378,1.149652,...,2,2,2,2,2,2,2,0,3,22
2,chr1,3440451,C,T,chr1,3440217,3443046,1.283403,1.104182,1.262983,...,0,0,0,0,0,0,0,21,4,0
3,chr1,3441472,T,G,chr1,3440217,3443046,1.283403,1.104182,1.262983,...,2,2,2,2,2,1,1,3,7,15
4,chr1,3441617,G,A,chr1,3440217,3443046,1.283403,1.104182,1.262983,...,0,1,0,0,0,0,0,21,4,0


In [3]:
snps_bed = pd.DataFrame()
snps_bed["chromosome"] = snps["CHROM"]
snps_bed["start"] = snps["POS"] - 1
snps_bed["end"] = snps["POS"]
snps_bed["name"] = snps["CHROM"] + ":" + snps["POS"].astype(str) + ":" + snps["REF"] + ":" + snps["ALT"]
snps_bed["score"] = 0
snps_bed["strand"] = "+"

snps_bed.head()

,chromosome,start,end,name,score,strand
0,chr1,2966571,2966572,chr1:2966572:G:T,0,+
1,chr1,2969790,2969791,chr1:2969791:A:G,0,+
2,chr1,3440450,3440451,chr1:3440451:C:T,0,+
3,chr1,3441471,3441472,chr1:3441472:T:G,0,+
4,chr1,3441616,3441617,chr1:3441617:G:A,0,+


In [4]:
!mkdir in_files

mkdir: cannot create directory ‘in_files’: File exists


In [5]:
snps_bed.to_csv("in_files/intergenic_enhancer_snps.bed", sep="\t", index=False, header=False)

In [6]:
%load_ext rpy2.ipython

ImportError: No module named rpy2.ipython

In [21]:
%%R 

library(motifbreakR)
library(BSgenome.Hsapiens.UCSC.hg38)
library(MotifDb)

snps.mb.frombed <- snps.from.file(file = "in_files/intergenic_enhancer_snps.bed", search.genome = BSgenome.Hsapiens.UCSC.hg38, format = "bed")

# IC

In [65]:
%%R
sessionInfo()

R version 3.6.3 (2020-02-29)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 16.04.7 LTS

Matrix products: default
BLAS:   /usr/lib/libblas/libblas.so.3.6.0
LAPACK: /usr/lib/lapack/liblapack.so.3.6.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=pl_PL.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=pl_PL.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=pl_PL.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=pl_PL.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
 [1] stats4    parallel  grid      tools     stats     graphics  grDevices
 [8] utils     datasets  methods   base     

other attached packages:
 [1] BSgenome.Hsapiens.UCSC.hg38_1.4.1 BSgenome_1.52.0                  
 [3] rtracklayer_1.44.2                GenomicRanges_1.36.0             
 [5] GenomeInfoDb_1.20.0               motifbreakR_1.14.0               
 [7] MotifDb_1.26.0               

In [22]:
%%R
results_ic <- motifbreakR(snpList = snps.mb.frombed, filterp = TRUE,
                       pwmList = hocomoco,
                       threshold = 1e-5,
                       method = "ic",
                       bkg = c(A=0.25, C=0.25, G=0.25, T=0.25),
                       BPPARAM = BiocParallel::bpparam())
results_ic.df = data.frame(results_ic)

In [23]:
results_ic_df = %Rget results_ic.df

In [24]:
%%R
results_log <- motifbreakR(snpList = snps.mb.frombed, filterp = TRUE,
                       pwmList = hocomoco,
                       threshold = 1e-5,
                       method = "log",
                       bkg = c(A=0.25, C=0.25, G=0.25, T=0.25),
                       BPPARAM = BiocParallel::bpparam())
results_log.df = data.frame(results_log)

In [25]:
results_log_df = %Rget results_log.df

In [26]:
%%R
results_default <- motifbreakR(snpList = snps.mb.frombed, filterp = TRUE,
                       pwmList = hocomoco,
                       threshold = 1e-5,
                       method = "default",
                       bkg = c(A=0.25, C=0.25, G=0.25, T=0.25),
                       BPPARAM = BiocParallel::bpparam())
results_default.df = data.frame(results_default)

In [27]:
results_default_df = %Rget results_default.df

In [30]:
strong_ic = results_ic_df[results_ic_df["effect"] == "strong"]
strong_log = results_log_df[results_log_df["effect"] == "strong"]
strong_default = results_default_df[results_default_df["effect"] == "strong"]
print len(strong_log), len(strong_default), len(strong_ic)

1080 1026 949


In [31]:
print len(strong_log["providerId"].unique()), len(strong_default["providerId"].unique()), len(strong_ic["providerId"].unique())

295 277 246


In [36]:
print len(set(strong_log["providerId"].unique()).intersection(set(strong_default["providerId"].unique())))
print len(set(strong_log["providerId"].unique()).intersection(set(strong_ic["providerId"].unique())))
print len(set(strong_default["providerId"].unique()).intersection(set(strong_ic["providerId"].unique())))

260
230
236


In [40]:
set([1,2,3] + [1,5,6,7])

{1, 2, 3, 5, 6, 7}

In [45]:
print "Jaccard log-default:"
intersection = len(set(strong_log["providerId"].unique()).intersection(set(strong_default["providerId"].unique())))
summ = len(set(list(strong_log["providerId"].unique()) + list(strong_default["providerId"].unique())))
print float(intersection) / summ

print "Jaccard log-ic:"
intersection = len(set(strong_log["providerId"].unique()).intersection(set(strong_ic["providerId"].unique())))
summ = len(set(list(strong_log["providerId"].unique()) + list(strong_ic["providerId"].unique())))
print float(intersection) / summ

print "Jaccard ic-default:"
intersection = len(set(strong_ic["providerId"].unique()).intersection(set(strong_default["providerId"].unique())))
summ = len(set(list(strong_ic["providerId"].unique()) + list(strong_default["providerId"].unique())))
print float(intersection) / summ

Jaccard log-default:
0.833333333333
Jaccard log-ic:
0.739549839228
Jaccard ic-default:
0.822299651568


In [46]:
strong_log.columns

Index([u'seqnames', u'start', u'end', u'width', u'strand', u'REF', u'ALT',
       u'snpPos', u'motifPos', u'geneSymbol', u'dataSource', u'providerName',
       u'providerId', u'seqMatch', u'pctRef', u'pctAlt', u'scoreRef',
       u'scoreAlt', u'Refpvalue', u'Altpvalue', u'alleleRef', u'alleleAlt',
       u'effect'],
      dtype='object')

In [47]:
strong_default.columns

Index([u'seqnames', u'start', u'end', u'width', u'strand', u'REF', u'ALT',
       u'snpPos', u'motifPos', u'geneSymbol', u'dataSource', u'providerName',
       u'providerId', u'seqMatch', u'pctRef', u'pctAlt', u'scoreRef',
       u'scoreAlt', u'Refpvalue', u'Altpvalue', u'alleleRef', u'alleleAlt',
       u'effect'],
      dtype='object')

In [50]:
log_default = pd.merge(strong_log, strong_default, on = [u'seqnames', u'start', u'end', u'width', u'strand', u'REF', u'ALT',
       u'snpPos', u'motifPos', u'geneSymbol', u'dataSource', u'providerName',
       u'providerId', u'seqMatch', u'alleleRef', u'alleleAlt',
       u'effect'], how = "outer", suffixes = ["_log", "_default"], indicator = True)
log_default.head()

,seqnames,start,end,width,strand,REF,ALT,snpPos,motifPos,geneSymbol,...,alleleRef,alleleAlt,effect,pctRef_default,pctAlt_default,scoreRef_default,scoreAlt_default,Refpvalue_default,Altpvalue_default,_merge
0,chr2,130820622,130820630,9,+,G,A,130820628,7,TFAP2A,...,0.904542,0.075047,strong,0.997011,0.858221,5.355083,4.609622,-2.147484e+09,-2.147484e+09,both
1,chr6,167700349,167700365,17,+,T,C,167700361,13,SPI1,...,0.834244,0.049019,strong,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2,chr16,87819636,87819647,12,+,G,T,87819640,5,ARNT2,...,0.055215,0.944785,strong,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,chr16,87819630,87819646,17,+,G,T,87819640,11,MAZ,...,0.981891,0.000000,strong,0.905329,0.792365,7.251340,6.346545,-2.147484e+09,-2.147484e+09,both
4,chr16,87819633,87819649,17,+,G,T,87819640,8,SP2,...,0.868132,0.000000,strong,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [57]:
log_default.loc[0]

seqnames                                        chr2
start                                      130820622
end                                        130820630
width                                              9
strand                                             +
REF                                                G
ALT                                                A
snpPos                                     130820628
motifPos                                           7
geneSymbol                                    TFAP2A
dataSource                                  HOCOMOCO
providerName                                 AP2A_f2
providerId                                AP2A_HUMAN
seqMatch             gcctggGgc                      
pctRef_log                                  0.997523
pctAlt_log                                  0.937036
scoreRef_log                                 9.49124
scoreAlt_log                                 7.00351
Refpvalue_log                           -2.147

In [55]:
results_default_df[(results_default_df["snpPos"] == 167700361) & (results_default_df["geneSymbol"] == "SPI1")]

,seqnames,start,end,width,strand,REF,ALT,snpPos,motifPos,geneSymbol,...,seqMatch,pctRef,pctAlt,scoreRef,scoreAlt,Refpvalue,Altpvalue,alleleRef,alleleAlt,effect


In [56]:
log_default[(log_default["_merge"] == "both")]

,seqnames,start,end,width,strand,REF,ALT,snpPos,motifPos,geneSymbol,...,alleleRef,alleleAlt,effect,pctRef_default,pctAlt_default,scoreRef_default,scoreAlt_default,Refpvalue_default,Altpvalue_default,_merge
0,chr2,130820622,130820630,9,+,G,A,130820628,7,TFAP2A,...,0.904542,0.075047,strong,0.997011,0.858221,5.355083,4.609622,-2.147484e+09,-2.147484e+09,both
3,chr16,87819630,87819646,17,+,G,T,87819640,11,MAZ,...,0.981891,0.000000,strong,0.905329,0.792365,7.251340,6.346545,-2.147484e+09,-2.147484e+09,both
5,chr16,87819625,87819648,24,+,G,T,87819640,16,SP4,...,0.956284,0.000000,strong,0.911389,0.824223,8.781447,7.941587,-2.147484e+09,-2.147484e+09,both
8,chr1,232704878,232704889,12,-,A,G,232704884,6,HLTF,...,0.000000,0.711111,strong,0.855328,0.953372,3.645878,4.063795,-2.147484e+09,-2.147484e+09,both
9,chr2,70563871,70563881,11,+,T,C,70563872,2,ZFHX3,...,1.000000,0.000000,strong,0.937418,0.808890,5.584066,4.818441,-2.147484e+09,-2.147484e+09,both
17,chr1,221996072,221996082,11,+,G,C,221996081,10,EGR2,...,0.855556,0.000000,strong,0.960548,0.859486,6.338987,5.672043,-2.147484e+09,-2.147484e+09,both
18,chr1,221996070,221996088,19,+,G,C,221996081,12,SP1,...,0.939936,0.009671,strong,0.954905,0.849870,7.862416,6.997591,-2.147484e+09,-2.147484e+09,both
19,chr1,221996075,221996085,11,+,G,C,221996081,7,SP1,...,0.929342,0.012164,strong,0.979365,0.858748,6.826437,5.985700,-2.147484e+09,-2.147484e+09,both
21,chr1,221996071,221996090,20,+,G,C,221996081,11,SP3,...,0.890211,0.034337,strong,0.940098,0.844021,7.160742,6.428918,-2.147484e+09,-2.147484e+09,both
23,chr1,221996073,221996085,13,+,G,C,221996081,9,WT1,...,1.000000,0.000000,strong,0.980612,0.839129,6.626335,5.670285,-2.147484e+09,-2.147484e+09,both


In [14]:
#results_df.to_csv("out_files/intergenic_enhancer_snps_motifbreakR_all.csv", sep = "\t", index=False)

In [15]:
len(results_df[results_df["effect"] == "strong"].groupby(["seqnames", "snpPos"]))

5789

In [16]:
strong_snps = results_df[results_df["effect"] == "strong"][["seqnames", "snpPos", "REF", "ALT"]].drop_duplicates()
strong_snps = strong_snps.rename(columns = {"seqnames":"CHROM", "snpPos":"POS"})
len(strong_snps)

5789

In [17]:
strong_snps.head()

,CHROM,POS,REF,ALT
1,chr1,2966572,G,T
23,chr1,67618872,C,T
26,chr1,114081245,C,T
38,chr1,175808465,T,C
52,chr1,203403058,A,G


# Calculate correlation with H3K27ac

In [18]:
snps.head()

,CHROM,POS,REF,ALT,chr,start,end,GB08.act,GB02.act,GB01.act,...,GB15.var,PA01.var,PA02.var,PA04.var,PA05.var,PA06.var,PG11.var,Num homref,Num het,Num homalt
0,chr1,2966572,G,T,chr1,2966403,2970363,0.985579,1.114378,1.149652,...,1,1,0,1,1,1,2,3,11,11
1,chr1,2969791,A,G,chr1,2966403,2970363,0.985579,1.114378,1.149652,...,2,2,2,2,2,2,2,0,3,22
2,chr1,3440451,C,T,chr1,3440217,3443046,1.283403,1.104182,1.262983,...,0,0,0,0,0,0,0,21,4,0
3,chr1,3441472,T,G,chr1,3440217,3443046,1.283403,1.104182,1.262983,...,2,2,2,2,2,1,1,3,7,15
4,chr1,3441617,G,A,chr1,3440217,3443046,1.283403,1.104182,1.262983,...,0,1,0,0,0,0,0,21,4,0


In [19]:
snps.columns

Index([u'CHROM', u'POS', u'REF', u'ALT', u'chr', u'start', u'end', u'GB08.act',
       u'GB02.act', u'GB01.act', u'PA04.act', u'PA01.act', u'PA02.act',
       u'GB04.act', u'GB05.act', u'DA03.act', u'GB06.act', u'DA04.act',
       u'GB07.act', u'DA05.act', u'GB03.act', u'DA01.act', u'DA06.act',
       u'DA01.var', u'DA02.var', u'DA03.var', u'DA04.var', u'DA05.var',
       u'DA06.var', u'DA07.var', u'GB01.var', u'GB02.var', u'GB03.var',
       u'GB04.var', u'GB05.var', u'GB06.var', u'GB07.var', u'GB08.var',
       u'GB10.var', u'GB12.var', u'GB14.var', u'GB15.var', u'PA01.var',
       u'PA02.var', u'PA04.var', u'PA05.var', u'PA06.var', u'PG11.var',
       u'Num homref', u'Num het', u'Num homalt'],
      dtype='object')

In [20]:
snps.loc[0]

CHROM             chr1
POS            2966572
REF                  G
ALT                  T
chr               chr1
start          2966403
end            2970363
GB08.act      0.985579
GB02.act       1.11438
GB01.act       1.14965
PA04.act       1.10058
PA01.act      0.694338
PA02.act      0.730377
GB04.act      0.920262
GB05.act      0.774194
DA03.act       1.00499
GB06.act      0.382496
DA04.act      0.908183
GB07.act      0.439584
DA05.act      0.620074
GB03.act      0.413482
DA01.act      0.351012
DA06.act      0.605413
DA01.var             2
DA02.var             2
DA03.var             1
DA04.var             2
DA05.var             2
DA06.var             2
DA07.var             2
GB01.var             2
GB02.var             1
GB03.var             2
GB04.var             2
GB05.var             0
GB06.var             2
GB07.var             1
GB08.var             1
GB10.var             1
GB12.var             1
GB14.var             0
GB15.var             1
PA01.var             1
PA02.var   

In [21]:
snps_changing_motifs = pd.merge(strong_snps, snps, how = "left", on = ["CHROM", "POS", "REF", "ALT"])
snps_changing_motifs.head()

,CHROM,POS,REF,ALT,chr,start,end,GB08.act,GB02.act,GB01.act,...,GB15.var,PA01.var,PA02.var,PA04.var,PA05.var,PA06.var,PG11.var,Num homref,Num het,Num homalt
0,chr1,2966572,G,T,chr1,2966403,2970363,0.985579,1.114378,1.149652,...,1,1,0,1,1,1,2,3,11,11
1,chr1,67618872,C,T,chr1,67617860,67619774,0.712716,0.653273,0.644964,...,0,0,0,0,1,0,1,17,8,0
2,chr1,114081245,C,T,chr1,114080440,114082945,0.456031,0.476802,0.533347,...,1,2,1,1,2,1,0,5,10,10
3,chr1,175808465,T,C,chr1,175808143,175811073,0.532068,0.450124,0.411408,...,2,1,1,2,1,1,2,0,9,16
4,chr1,203403058,A,G,chr1,203402080,203403501,0.854990,0.935986,0.863218,...,0,0,1,1,1,0,0,8,13,4


In [22]:
len(snps_changing_motifs)

5789

In [23]:
samples_with_h3k27ac = [col.split('.')[0] for col in snps_changing_motifs.columns if ".act" in col]

In [24]:
def perform_spearmanr_coverage(row):
    variant_columns = [col for col in row.index if '.var' in col]
    homref_indices = [i for i,val in enumerate(row[variant_columns]) if val != './.' and float(val) == 0]
    het_indices = [i for i,val in enumerate(row[variant_columns]) if val != './.' and float(val) == 1]
    homalt_indices = [i for i,val in enumerate(row[variant_columns]) if val != './.' and float(val) == 2]
    
    homref_patients = set([variant_columns[i].split('.')[0] for i in homref_indices]).intersection(samples_with_h3k27ac)
    het_patients = set([variant_columns[i].split('.')[0] for i in het_indices]).intersection(samples_with_h3k27ac)
    homalt_patients = set([variant_columns[i].split('.')[0] for i in homalt_indices]).intersection(samples_with_h3k27ac) 
    
    homref_activity_columns = [p + '.act' for p in homref_patients]
    het_activity_columns = [p + '.act' for p in het_patients]
    homalt_activity_columns = [p + '.act' for p in homalt_patients]

    activity_vector = list(row[homref_activity_columns].values) + list(row[het_activity_columns].values) + list(row[homalt_activity_columns].values)
    genotype_vector = [0]*len(homref_patients) + [1]*len(het_patients) + [2]*len(homalt_patients)
    #print activity_vector, genotype_vector
    rho, pval = spearmanr(activity_vector, genotype_vector)
    num_samples = len(activity_vector)
    return pd.Series([rho, pval, num_samples], index=['enh_corr_coeff', 'enh_pval', 'Number of samples with data'])

In [25]:
snps_changing_motifs[['h3k27ac_coeff', 'h3k27ac_pval', 'Number of samples with data']] = snps_changing_motifs.apply(perform_spearmanr_coverage, axis=1)

/home/magda/.local/lib/python2.7/site-packages/numpy/lib/function_base.py:3162: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/magda/.local/lib/python2.7/site-packages/numpy/lib/function_base.py:3163: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/home/magda/.local/lib/python2.7/site-packages/scipy/stats/_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/home/magda/.local/lib/python2.7/site-packages/scipy/stats/_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/home/magda/.local/lib/python2.7/site-packages/scipy/stats/_distn_infrastructure.py:1831: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


In [26]:
snps_changing_motifs.head()

,CHROM,POS,REF,ALT,chr,start,end,GB08.act,GB02.act,GB01.act,...,PA04.var,PA05.var,PA06.var,PG11.var,Num homref,Num het,Num homalt,h3k27ac_coeff,h3k27ac_pval,Number of samples with data
0,chr1,2966572,G,T,chr1,2966403,2970363,0.985579,1.114378,1.149652,...,1,1,1,2,3,11,11,-0.317620,0.230616,16.0
1,chr1,67618872,C,T,chr1,67617860,67619774,0.712716,0.653273,0.644964,...,0,1,0,1,17,8,0,0.031311,0.908356,16.0
2,chr1,114081245,C,T,chr1,114080440,114082945,0.456031,0.476802,0.533347,...,1,2,1,0,5,10,10,-0.217214,0.419036,16.0
3,chr1,175808465,T,C,chr1,175808143,175811073,0.532068,0.450124,0.411408,...,2,1,1,2,0,9,16,-0.248633,0.353120,16.0
4,chr1,203403058,A,G,chr1,203402080,203403501,0.854990,0.935986,0.863218,...,1,1,0,0,8,13,4,-0.266474,0.318461,16.0


In [27]:
# no pvalue for some samples because no H3K27ac is available for variants samples
snps_changing_motifs_h3k27ac = snps_changing_motifs[~snps_changing_motifs["h3k27ac_pval"].isnull()]
print len(snps_changing_motifs), len(snps_changing_motifs_h3k27ac)

5789 5740


In [28]:
multipletests_correction = multipletests(snps_changing_motifs_h3k27ac['h3k27ac_pval'], alpha=0.1, 
                  method='fdr_bh', is_sorted=False, returnsorted=False)

snps_changing_motifs_h3k27ac['h3k27ac_B-H_reject_H0'] = multipletests_correction[0]
snps_changing_motifs_h3k27ac['h3k27ac_corr_pval'] = multipletests_correction[1]
len(snps_changing_motifs_h3k27ac[snps_changing_motifs_h3k27ac["h3k27ac_B-H_reject_H0"]])

/home/magda/.local/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/magda/.local/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


0

In [29]:
snps_changing_motifs_h3k27ac.sort_values(by = "h3k27ac_pval")[:20]

,CHROM,POS,REF,ALT,chr,start,end,GB08.act,GB02.act,GB01.act,...,PA06.var,PG11.var,Num homref,Num het,Num homalt,h3k27ac_coeff,h3k27ac_pval,Number of samples with data,h3k27ac_B-H_reject_H0,h3k27ac_corr_pval
2622,chr2,204276082,G,A,chr2,204275182,204276466,0.886374,0.781975,0.797349,...,2,1,5,11,9,0.791444,0.000260,16.0,False,0.597999
3613,chr11,112904101,A,G,chr11,112901289,112904861,0.823179,0.969858,0.786771,...,0,2,16,5,4,-0.747882,0.000864,16.0,False,0.597999
180,chr6,149108310,G,T,chr6,149106809,149109381,0.825277,0.587317,0.808107,...,0,1,6,15,4,-0.731954,0.001267,16.0,False,0.597999
30,chr6,149107861,A,G,chr6,149106809,149109381,0.825277,0.587317,0.808107,...,0,1,6,15,4,-0.731954,0.001267,16.0,False,0.597999
5225,chr6,1526818,C,T,chr6,1525125,1527848,1.056928,0.896662,0.999428,...,0,2,3,18,4,0.728190,0.001381,16.0,False,0.597999
2116,chr6,22709254,A,G,chr6,22708456,22711699,0.897039,0.411306,0.710636,...,2,1,5,15,5,-0.722713,0.001563,16.0,False,0.597999
2564,chr6,22710795,A,G,chr6,22708456,22711699,0.897039,0.411306,0.710636,...,2,1,5,15,5,-0.722713,0.001563,16.0,False,0.597999
2490,chr6,22709812,A,C,chr6,22708456,22711699,0.897039,0.411306,0.710636,...,2,1,5,15,5,-0.722713,0.001563,16.0,False,0.597999
1964,chr6,22709201,A,G,chr6,22708456,22711699,0.897039,0.411306,0.710636,...,2,1,5,15,5,-0.722713,0.001563,16.0,False,0.597999
2415,chr6,22709510,C,G,chr6,22708456,22711699,0.897039,0.411306,0.710636,...,2,1,5,15,5,-0.722713,0.001563,16.0,False,0.597999
